In [6]:
import pandas as pd
import re
from io import StringIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

In [7]:
def preprocessing(data):
    # data = re.sub(' {3,}', '@', data)
    # data = re.sub(' ', "", data)
    # data = re.sub('@', ' ', data)
    data = re.sub(r'\n', '', data)
    REMOVE_CHARS = re.compile("'+|(=+.{2,30}=+)|__TOC__|(ファイル:).+|:(en|de|it|fr|es|kr|zh|no|fi):", re.UNICODE)
    SPACE_CHARS = re.compile("(\\s|゙|゚|　)+", re.UNICODE)
    EMAIL_PATTERN = re.compile("(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", re.UNICODE)
    URL_PATTERN = re.compile("(ftp|http|https)?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", re.UNICODE)
    REMOVE_TOKEN_CHARS = re.compile("(\\*$|:$|^파일:.+|^;)", re.UNICODE)
    MULTIPLE_SPACES = re.compile(' +', re.UNICODE)
    EXCEPT_KOREAN = re.compile("[^ ㄱ-ㅣ가-힣+|.]", re.UNICODE)
    data = re.sub(EMAIL_PATTERN, ' ', data)  # remove email pattern
    data = re.sub(URL_PATTERN, ' ', data) # remove url pattern
    data = re.sub(REMOVE_CHARS, ' ', data)  # remove unnecessary chars
    data = re.sub(EXCEPT_KOREAN, ' ', data)
    data = re.sub(SPACE_CHARS, ' ', data)
    data = re.sub(MULTIPLE_SPACES, ' ', data)
    data = re.sub(' [.]', '', data)
    stop_words = "등 등이 에 에서 와 과 은 는 의 개 개의 년간 로 을 를 하는 총 월 목표주가 현재주가 연결 요약 재무제표 십억원 만주 주 주가 배 년 일 만 억원 우 좌 원 억 백만 약 각각 으로 십 액면가 종가 자본금 발행주식수 시가총액 외국인지분율 일평균거래량 일평균거래대금 주가수익률 절대수익률 상대수익률 배당수익률"
    stop_words = set(stop_words.split(' '))
    result = [word for word in data.split(' ') if not word in stop_words]
    result = " ".join(result)
    return result

In [8]:
def read_pdf_PDFMINER(pdf_file_path):
    """
    pdf_file_path: 'dir/aaa.pdf'로 구성된 path로부터 
    내부의 text 파일을 모두 읽어서 스트링을 리턴함.
    https://pdfminersix.readthedocs.io/en/latest/tutorials/composable.html
    """
    output_string = StringIO()
    with open(pdf_file_path, 'rb') as f:
        parser = PDFParser(f)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for pageNumber, page in enumerate(PDFPage.create_pages(doc)):
            if pageNumber == 0:
                interpreter.process_page(page)
                break

    return str(output_string.getvalue())

### Google 번역기 활용
### pip install googletrans==4.0.0-rc1로 설치할 것

In [107]:
import googletrans


def back_translate_google(source_text):
    translator = googletrans.Translator()
    
    # en와 ja 비슷, zh-cn와 zh-tw 비슷
    lang_list = ["en", "zh-cn"]
    back_trans_list = list()
    
    # print(googletrans.LANGUAGES)
    for lang in lang_list:
        trans_text = translator.translate(source_text, dest=lang)
        back_trans_text = translator.translate(trans_text.text, dest="ko")
        back_trans_list.append(back_trans_text.text)
    
    return back_trans_list
    

### Naver 파파고 활용

In [116]:
import os
import sys
import urllib
import json


def back_translate_papago(source_text):
    # 보안상 개인 naver api key 사용 바람
    json_file = open("./translator_key.json", encoding="utf-8")
    key_dict = json.loads(json_file.read())

    client_id = key_dict["client_id"]
    client_secret = key_dict["client_secret"]

    url = "https://openapi.naver.com/v1/papago/n2mt"

    lang_list = ["en", "ja"]
    
    target_lang = lang_list[0]
    ### 한국어 -> 외국어
    enc_text = urllib.parse.quote(source_text)
    data = f"source=ko&target={target_lang}&text=" + enc_text
    
    # 요청 header 및 parameter
    req_header = {"X-Naver-Client-Id":client_id, "X-Naver-Client-Secret":client_secret}
    req_parameter = {"source":"ko", "target":target_lang, "text":encText}
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()

    if(rescode == 200):
        response_body = response.read()
        res_json = json.loads(response_body.decode("utf-8"))
        trans_text = res_json["message"]["result"]["translatedText"]
    else:
        print("error Code:" + rescode)
        sys.exit(1)
    
    ### 외국어 -> 한국어
    enc_text = urllib.parse.quote(trans_text)
    data = f"source={target_lang}&target=ko&text=" + enc_text
    
    # 요청 header 및 parameter
    req_parameter = {"source":target_lang, "target":"ko", "text":trans_text}
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()

    if(rescode == 200):
        response_body = response.read()
        res_json = json.loads(response_body.decode("utf-8"))
        back_trans_text = res_json["message"]["result"]["translatedText"]
    else:
        print("error Code:" + rescode)
        sys.exit(1)
    
    print(source_text)
    print("-" * 30)
    print(trans_text)
    print("-" * 30)
    print(back_trans_text)
    
    return back_trans_text


In [117]:
def main():
    pdf_file_path = "./data/test_pdf_file1.pdf"
    text_data = read_pdf_PDFMINER(pdf_file_path)
    preprocessed_text_data = preprocessing(text_data)
    
#     back_trans_list = back_translate_google(preprocessed_text_data)
    
#     print(preprocessed_text_data)
#     print("-" * 50)
#     for document in back_trans_list:
#         print(document)
#         print("-" * 50)

    back_trans_text = back_translate_papago(preprocessed_text_data)
    
    
if __name__ == "__main__":
    main()

 음식료 담배 외형성장 원가부담 매일유업 하향 상승여력 매출액 영업이익 순이익 별도 음식료 담배 이경신 박이경 매일유업의 매출액과 영업이익은 + 예상한다. 주요제품 성장세는 여타업체대비 견조한 상황으로 시장지배력 또한 유의미하다. 다만 대외환경 악화에 따른 원재료비용 상승으로 단기 영업이익은 다소 눈높이를 낮출 필요가 있으나 향후 부담 축소시 추가 영업실적 확대 가능성을 열어둔다. 냉장유통이 필요한 전통적인 유제품대비 온라인 배송에 상대적인 경쟁우위 요소를 갖춘 테트라팩 수요확대와 관련된 카테고리의 외형성장이 꾸준하며 최근 성장세가 견조한 곡물음료 상하목장 흐름은 이익기여 측면에서도 긍정적이다. 컵커피 또한 가격인상 효과가 더해져 높은 성장세를 시현할 것으로 판단한다. 다만 원가 상승과 관련한 부담이 동시에 반영되면서 일부 품목에서의 마진하락이 예상되며 추가적으로 원유가격의 물가 연동제와 관련한 가격결정에 따라 이후 원가 변동 가능성이 존재한다. 그러나 어려운 대외환경에서도 여타 주요제품의 시장지배력 관련 변동성은 낮다는 점에서 부담 축소시 영업실적 개선 반영을 기대하는데 무리가 없다. 투자의견 원으로 하향 코로나 및 출산율 저하와 관련된 대외변수 악화에도 불구 견고한 이익률이 유지되는 펀더멘털의 영향은 여타 동종업체대비 낮은 상황이다. 다만 대외변수 악화에 따른 비용부담이 영업실적에 반영됨을 감안한 추정치 변경으로 목표주가를 기존 만원에서 원으로 하향조정한다. 그러나 단기 일부 눈높이 조정에도 시장지배력 및 베이스효과를 고려한다면 부담축소 이후 영업실적 개선세가 꾸준히 이어질 가능성에 무게를 실으며 향후 외부변수 개선시 추가 저변확대 가능성을 감안 높은 밸류에이션 매력도가 고려된 매수접근이 유효할 전망이다. 단위 매출액 영업이익 영업이익률 세전이익 세전이익률 자료 하이투자증권 컨센서스는 기준 매일유업 
------------------------------
Food, beverage, tobacco, external growth, cost burden, do

In [1]:
from konlpy.tag import Mecab
tokenizer = Mecab()
tokenizer.morphs(data)

NameError: name 'data' is not defined

In [161]:
from transformers import AutoModel, AutoTokenizer

# Load Tokenizer and Model
kb_albert_model_path = "/home/piai/다운로드/kb-albert-char-base-v2"
pt_model = AutoModel.from_pretrained(kb_albert_model_path)
tokenizer = AutoTokenizer.from_pretrained(kb_albert_model_path)

# inference text input to sentence vector of last layer
pt_inputs = tokenizer(data, return_tensors='pt')
pt_outputs = pt_model(**pt_inputs)[0]
print(pt_outputs)

Some weights of the model checkpoint at /home/piai/다운로드/kb-albert-char-base-v2 were not used when initializing AlbertModel: ['predictions.dense.weight', 'predictions.LayerNorm.bias', 'predictions.decoder.bias', 'predictions.dense.bias', 'predictions.LayerNorm.weight', 'sop_classifier.classifier.bias', 'predictions.bias', 'sop_classifier.classifier.weight', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Token indices sequence length is longer than the specified maximum sequence length for this model (877 > 512). Running this sequence through the model wil

RuntimeError: The size of tensor a (877) must match the size of tensor b (512) at non-singleton dimension 1

In [20]:
pt_inputs["input_ids"]

tensor([[   2, 4833, 6659, 4209, 6263, 6423, 6483, 6338, 6659, 6491, 5247, 6367,
         6362, 6453, 4755, 6361, 4150, 6755, 6274, 6275, 4732, 6279, 6361, 5415,
         6608, 4877, 6275, 6363, 6266, 6275, 6474, 6352, 4521, 6378, 6363, 4286,
         6429, 6483, 6338, 4207, 5415, 7025, 6258, 6659, 4209, 6263, 6423, 6493,
         6314, 6869, 5369, 6436, 6341, 6271, 4877, 6446, 6685, 4094, 6483, 4779,
         6875, 6368, 6263, 3774, 6394, 5371, 6338, 6659, 6491, 6377, 5197, 6653,
         6298, 6357, 5056, 6626, 4667, 6293, 6577, 7025, 4207, 4877, 6499, 4910,
         6603, 5415, 7025, 6377, 4638, 6558, 6263, 6419, 6298, 6321, 6537, 4286,
         6429, 5371, 6338, 6259, 3449, 6334, 6483, 4779, 6989, 4421, 6347, 6263,
         4672, 6284, 6432, 6259, 3815, 6271, 6259, 4514, 6458, 4833, 4398, 6292,
         6422, 6360, 5433, 6379, 6271, 6259, 4094, 6295, 6415, 6684, 5083, 6396,
         6258, 5063, 6266, 6271, 4776, 6483, 4107, 6506, 4421, 6347, 6341, 6360,
         5375, 6298, 6436, 6

In [24]:
tokenizer.convert_ids_to_tokens(pt_inputs["input_ids"][0])

['[CLS]',
 '제',
 '##약',
 '바',
 '##이',
 '##오',
 '##한',
 '##미',
 '##약',
 '##품',
 '투',
 '##자',
 '##의',
 '##견',
 '유',
 '##지',
 '목',
 '##표',
 '##주',
 '##가',
 '원',
 '##유',
 '##지',
 '현',
 '##재',
 '주',
 '##가',
 '##원',
 '##시',
 '##가',
 '##총',
 '##액',
 '십',
 '##억',
 '##원',
 '북',
 '##경',
 '##한',
 '##미',
 '및',
 '현',
 '##황',
 '##제',
 '##약',
 '바',
 '##이',
 '##오',
 '##권',
 '##해',
 '##순',
 '하',
 '##반',
 '##기',
 '##에',
 '주',
 '##목',
 '##할',
 '만',
 '##한',
 '이',
 '##슈',
 '##들',
 '##이',
 '다',
 '##수',
 '한',
 '##미',
 '##약',
 '##품',
 '##을',
 '탐',
 '##방',
 '##하',
 '##여',
 '최',
 '##근',
 '영',
 '##업',
 '##현',
 '##황',
 '및',
 '주',
 '##요',
 '진',
 '##행',
 '현',
 '##황',
 '##을',
 '업',
 '##데',
 '##이',
 '##트',
 '##하',
 '##였',
 '##음',
 '북',
 '##경',
 '한',
 '##미',
 '##는',
 '견',
 '##조',
 '##한',
 '이',
 '##익',
 '성',
 '##장',
 '##이',
 '예',
 '##상',
 '##되',
 '##는',
 '데',
 '##에',
 '##는',
 '시',
 '##럽',
 '제',
 '생',
 '##산',
 '##라',
 '##인',
 '확',
 '##대',
 '##에',
 '##는',
 '만',
 '##성',
 '##질',
 '##환',
 '치',
 '##료',
 '##제',
 '출',
 '##시',


In [3]:
information = pd.read_csv('information.csv')
information.tail()

,date,title,price,consensus,analyst,firm,report_idx
9016,2021-05-10,SPC삼립(005610)증진된 체력에 주목을,"90,000",Buy,한유정,대신증권,589078
9017,2021-05-10,한섬(020000)1Q21 Review: 깜놀,"55,000",Buy,하누리,메리츠증권,589077
9018,2021-05-10,CJ대한통운(000120)이제부터가 본 게임,"250,000",Buy,"양지환, 이지수",대신증권,589072
9019,2021-05-10,에코프로비엠(247540)양극재는 역시 에코프로비엠,"240,000",Buy,이종형,키움증권,589068
9020,2021-05-10,롯데케미칼(011170)실적 턴어라운드 개시,"460,000",Buy,이동욱,키움증권,589066
